In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports

from GEMS_TCO import kernels_month

from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, BaseLogger
from GEMS_TCO import kernels_columns as kernels_reparam_space_time_gpu_col
from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

Load daily data applying max-min ordering

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

#lat_range_input=[-3,2]      
#lon_range_input=[121, 131] 
lat_range_input=[-3,-1]      
lon_range_input=[121, 125] 

df_map, ord_mm, nns_map, day_offsets = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,

lat_range=lat_range_input,   
lon_range=lon_range_input
  
)

--- Global Monthly Mean for 2024-7: 257.0667 ---


In [3]:
daily_aggregated_tensors_dw = [] 
daily_hourly_maps_dw = []      

daily_aggregated_tensors_vecc = [] 
daily_hourly_maps_vecc = []   

for day_index in range(31):
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    hour_indices = [hour_start_index, hour_end_index]

    # --- DW용 데이터 로드 (day_offsets 인자 추가) ---
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        day_offsets,  # <--- 이 부분이 추가되어야 합니다
        hour_indices, 
        ord_mm=ord_mm,
        dtype=torch.float64, 
        keep_ori=False
    )
    daily_aggregated_tensors_dw.append(day_aggregated_tensor)
    daily_hourly_maps_dw.append(day_hourly_map)

    # --- Vecchia용 데이터 로드 (day_offsets 인자 추가) ---
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        day_offsets,  # <--- 이 부분이 추가되어야 합니다
        hour_indices, 
        ord_mm=ord_mm,
        dtype=torch.float64, 
        keep_ori= True
    )
    daily_aggregated_tensors_vecc.append(day_aggregated_tensor)
    daily_hourly_maps_vecc.append(day_hourly_map)

print(f"Aggregated Tensor Shape: {daily_aggregated_tensors_vecc[0].shape}")
# 예상 출력: torch.Size([행수, 12]) -> 열이 12개여야 성공입니다.

Aggregated Tensor Shape: torch.Size([22680, 11])


In [5]:
v=0.5

patience, factor = 5, 0.5
# [수정 2] 이웃 숫자 확인 (A+B+C = 26개 이상이어야 함)
mm_cond_number = 8        # 넉넉하게 설정
nheads = 15                # 시간당 30개 (앞부분 Low Freq)

import torch
import numpy as np

# ==========================================
# 1. 병합할 기간 설정 (User Setting)
# ==========================================
# 예: 0일부터 9일까지 (총 10일치)만 학습하고 싶다면 range(10)
# 예: 31일 전체를 학습하고 싶다면 range(31)
TARGET_DAYS_IDX = range(28) 

print(f"Merging data for days: {list(TARGET_DAYS_IDX)}")

# ==========================================
# 2. Tensor 병합 (Torch.cat)
# ==========================================
# 선택한 날짜들의 Tensor만 리스트에서 가져옵니다.
selected_tensors = [daily_aggregated_tensors_vecc[i] for i in TARGET_DAYS_IDX]

# dim=0 (행 방향)으로 이어 붙입니다.
full_aggregated_tensor_vecc = torch.cat(selected_tensors, dim=0)

# ==========================================
# 3. Input Map 병합 (Dict Update)
# ==========================================
full_input_map_vecc = {}

# 선택한 날짜들의 Dictionary를 순회하며 하나로 합칩니다.
# 주의: load_working_data에서 생성된 key(시간 인덱스)가 날짜별로 겹치지 않고
# global index(예: 0~7, 8~15...)로 되어 있어야 덮어씌워지지 않습니다.
# 작성하신 코드의 hour_indices 계산식을 보면 Global Index이므로 안전합니다.
for i in TARGET_DAYS_IDX:
    daily_map = daily_hourly_maps_vecc[i]
    full_input_map_vecc.update(daily_map)

# ==========================================
# 4. NNS Map 병합 (가장 중요 ★)
# ==========================================
# 주의: NNS Map(이웃 인덱스)이 만약 '하루 단위'로 0부터 시작하게 계산되어 있다면,
# 2일차의 이웃 인덱스는 1일차 데이터 개수만큼 더해줘야 합니다 (Offset).
# 만약 NNS Map을 아직 안 만드셨거나, 전체 데이터 기준 새로 만드실 거라면 이 부분은 패스하세요.

# (예시: 만약 daily_nns_maps 라는 리스트가 있다고 가정할 때의 병합 로직)
# full_nns_map = []
# current_offset = 0
# for i in TARGET_DAYS_IDX:
#     # 해당 날짜의 데이터 개수
#     day_len = daily_aggregated_tensors_vecc[i].shape[0]
#     
#     # 해당 날짜의 NNS Map을 가져와서 Offset 더하기 (유효한 인덱스만)
#     # nns_local = daily_nns_maps[i] + current_offset
#     # full_nns_map.extend(nns_local)
#     
#     current_offset += day_len

# ==========================================
# 5. 결과 확인
# ==========================================
print("-" * 30)
print(f"✅ Merging Complete for {len(TARGET_DAYS_IDX)} days.")
print(f"Tensor Shape: {full_aggregated_tensor_vecc.shape}")
print(f"Map Keys (Hours): {len(full_input_map_vecc)} hours")
print("-" * 30)

# 이제 모델에는 아래 변수를 넣으시면 됩니다.
# aggregated_data = full_aggregated_tensor_vecc
# input_map = full_input_map_vecc

Merging data for days: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
------------------------------
✅ Merging Complete for 28 days.
Tensor Shape: torch.Size([635040, 11])
Map Keys (Hours): 224 hours
------------------------------


In [6]:
import torch
import numpy as np
import time

# --- 1. Global Settings ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# [설정] 최적화 파라미터 (10일치 데이터를 위해 Step 수 상향)
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 3       # [수정] 수렴을 위해 3 -> 50 권장 (디버깅용이면 3 유지)
LBFGS_HISTORY_SIZE = 100   
LBFGS_MAX_EVAL = 100        # Line search 횟수

# [설정] 모델링할 기간 선택 (예: 앞의 10일만 사용)
# daily_aggregated_tensors_vecc 리스트의 인덱스 기준
TARGET_DAYS_IDX = range(28) # 0일차 ~ 9일차

# --- 2. Data Preparation (Merge Lists into One) ---
print(f'\n{"="*40}')
print(f'--- Merging Data for Days: {list(TARGET_DAYS_IDX)} ---')
print(f'{"="*40}')

# (1) Tensor 합치기 (Time 방향으로 연결)
# daily_aggregated_tensors_vecc는 [Tensor(Day1), Tensor(Day2)...] 형태
selected_tensors = [daily_aggregated_tensors_vecc[i] for i in TARGET_DAYS_IDX]
full_aggr_data = torch.cat(selected_tensors, dim=0).to(DEVICE)

# (2) Input Map 합치기 (Dictionary Update)
# daily_hourly_maps_vecc는 [Dict(Day1), Dict(Day2)...] 형태
full_input_map = {}
for i in TARGET_DAYS_IDX:
    # 시간 인덱스(Key)가 겹치지 않는다고 가정 (Global Index 사용 시 안전)
    full_input_map.update(daily_hourly_maps_vecc[i])

# (3) NNS Map 설정
# [가정] nns_map은 공간상의 이웃(Spatial NNS)만 정의되어 있음.
# 모델 내부에서 시간 Offset을 자동으로 더해주므로, 그대로 사용하면 됨.
full_nns_map = nns_map # (User Context에서 이미 주어진 변수)

print(f"Merged Tensor Shape: {full_aggr_data.shape}")
print(f"Merged Map Keys: {len(full_input_map)}")


# --- 3. Initial Parameters ---
init_sigmasq   = 13.059
init_range_lat = 0.2
init_range_lon = 0.25
init_advec_lat = 0.0218
init_range_time = 1.5
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map parameters to Log-Scale
init_phi2 = 1.0 / init_range_lon                
init_phi1 = init_sigmasq * init_phi2            
init_phi3 = (init_range_lon / init_range_lat)**2  
init_phi4 = (init_range_lon / init_range_time)**2      

initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]


# --- 4. Instantiate & Fit ---
print(f"\nInstantiating SpatioTemporalModel (Fit strategy: A+B+C)...")

# [중요] 정의해둔 Class 이름 확인 (여기서는 fit_vecchia_lbfgs로 가정)
# 만약 import kernels_reparam_space_time 했다면 앞에 모듈명 붙여야 함
model_instance = kernels_month.fit_vecchia_lbfgs( 
    smooth = 0.5,                    # Smoothness (v) 값 (User Context 확인 필요)
    input_map = full_input_map,      # [수정] 합쳐진 데이터
    aggregated_data = full_aggr_data,# [수정] 합쳐진 데이터
    nns_map = full_nns_map,          # [유지] 공간 NNS
    mm_cond_number = mm_cond_number, # (User Context 변수) 예: 30~40
    nheads = nheads                  # (User Context 변수) 예: 30
)

# Optimizer 설정
optimizer_vecc = model_instance.set_optimizer(
    params_list,     
    lr=LBFGS_LR,            
    max_iter=LBFGS_MAX_EVAL,        
    history_size=LBFGS_HISTORY_SIZE 
)

print(f"\n--- Starting Optimization (Full 10 Days) ---")
start_time = time.time()

# 실행
out, steps_ran = model_instance.fit_vecc_lbfgs(
    params_list,
    optimizer_vecc,
    max_steps=LBFGS_MAX_STEPS, 
    grad_tol=1e-7
)

end_time = time.time()
print(f"Optimization finished in {end_time - start_time:.2f}s.")
print(f"Final Params & Loss: {out}")


Using device: cpu

--- Merging Data for Days: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27] ---
Merged Tensor Shape: torch.Size([635040, 11])
Merged Map Keys: 224

Instantiating SpatioTemporalModel (Fit strategy: A+B+C)...

--- Starting Optimization (Full 10 Days) ---
🚀 Pre-computing (Top 15 Max-Min Ordered Heads per hour)... [Mean Lat: -2.0043] ✅ Done. (Total Heads: 3360, Total Tails: 631680)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/3 / Loss: 1.247805 ---
  Param 0: Value=3.4137, Grad=5.892208193798753e-07
  Param 1: Value=0.7938, Grad=-2.654627065246779e-07
  Param 2: Value=0.2175, Grad=-2.886890257192976e-07
  Param 3: Value=-3.2836, Grad=7.534578038521575e-10
  Param 4: Value=0.0050, Grad=8.544382242470401e-08
  Param 5: Value=-0.0822, Grad=6.400916973433972e-09
  Param 6: Value=-0.3507, Grad=5.398397513929642e-07
  Max Abs Grad: 5.892208e-07
------------------------------
--- Step 2/3 / Loss: 1.221592 --

In [ ]:
irr. (-3,2 121,131, intercept, time dummies, latitude)

Final Interpretable Params: {
    
'sigma_sq': 14.557378105972255, 
 'range_lon': 0.36297285215444397, 'range_lat': 0.33986461719454436, 'range_time': 1.8307357609214716, 'advec_lat': -0.0014072484629318405, 'advec_lon': -0.16623085410420968, 'nugget': 0.24999425486908594}
Vecchia Optimization finished in 67.75s. Results: [3.691525186381127, 1.0134272349673297, 0.13156138084623087, -3.2362903523329813, -0.0014072484629318405, -0.16623085410420968, -1.3863173419076031, 0.9065025151721705]

regular (hide nugget)
------------------------------
------------------------------
Final Interpretable Params: {'sigma_sq': 13.881823879707401, 'range_lon': 0.3308191657110315, 'range_lat': 0.3048282266752421, 'range_time': 1.6460269337530522, 'advec_lat': -0.0014028377727820455, 'advec_lon': -0.15541841240605433, 'nugget': 1.112667285815519e-07}
Vecchia Optimization finished in 141.61s. Results: [3.7367637299362526, 1.1061833800531369, 0.16364694440410635, -3.209095690658186, -0.0014028377727820455, -0.15541841240605433, -16.01133555793201, 0.9490311091657573]


irr  ar(1) offset logic 이용했을때 사실 차이 없고 latitude centering 했을 때 소수점 10쨰자리에서 살짝 개선



SyntaxError: invalid decimal literal (2978187764.py, line 7)